In [611]:
import os 
import pandas as pd
import numpy as np

path = os.getcwd()

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
le = LabelEncoder()

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

import seaborn as sns
import matplotlib.pyplot as plt

In [612]:
path = 'C:\\Users\\sunil\\Projects\\Dockship\\segmind_grand_ai_challenge_2021-dataset'

In [613]:
train = pd.read_csv(path + "\\dataset\\TRAIN.csv")
test = pd.read_csv(path + "\\dataset\\TEST.csv")
ss = pd.read_csv(path + "\\dataset\\sample_submission.csv")
shop = pd.read_csv(path + "\\dataset\\store.csv")

C:\Users\sunil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [614]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month
train['day'] = train['Date'].dt.day
train['dayofweek'] = train['Date'].dt.dayofweek
train['week'] = train['Date'].dt.week

test['year'] = test['Date'].dt.year
test['month'] = test['Date'].dt.month
test['day'] = test['Date'].dt.day
test['dayofweek'] = test['Date'].dt.dayofweek
test['week'] = test['Date'].dt.week

train['StateHoliday'].replace({0 : 0, '0':0, 'a':1, 'b':2, 'c':3}, inplace = True)
test['StateHoliday'].replace({0 : 0, '0':0, 'a':1, 'b':2, 'c':3}, inplace = True)

In [615]:
# Merging

store_type = dict(zip(shop['Store'], shop['StoreType']))
store_ass = dict(zip(shop['Store'], shop['Assortment']))
store_com = dict(zip(shop['Store'], shop['CompetitionDistance']))
store_month = dict(zip(shop['Store'], shop['CompetitionOpenSinceMonth']))
store_year = dict(zip(shop['Store'], shop['CompetitionOpenSinceYear']))
store_p2 = dict(zip(shop['Store'], shop['Promo2']))
store_pweek = dict(zip(shop['Store'], shop['Promo2SinceWeek']))
store_pyear = dict(zip(shop['Store'], shop['Promo2SinceYear']))
store_pi = dict(zip(shop['Store'], shop['PromoInterval']))

train['Store_Type'] = train['Store'].map(store_type)
train['Store_Assortment'] = train['Store'].map(store_ass)
train['Store_distance'] = train['Store'].map(store_com)
train['Store_CompetitionOpenSinceMonth'] = train['Store'].map(store_month)
train['Store_CompetitionOpenSinceYear'] = train['Store'].map(store_year)
train['Store_Promo2'] = train['Store'].map(store_p2)
train['Store_Promo2SinceWeek'] = train['Store'].map(store_pweek)
train['Store_Promo2SinceYear'] = train['Store'].map(store_pyear)
train['Store_PromoInterval'] = train['Store'].map(store_pi)


test['Store_Type'] = test['Store'].map(store_type)
test['Store_Assortment'] = test['Store'].map(store_ass)
test['Store_distance'] = test['Store'].map(store_com)
test['Store_CompetitionOpenSinceMonth'] = test['Store'].map(store_month)
test['Store_CompetitionOpenSinceYear'] = test['Store'].map(store_year)
test['Store_Promo2'] = test['Store'].map(store_p2)
test['Store_Promo2SinceWeek'] = test['Store'].map(store_pweek)
test['Store_Promo2SinceYear'] = test['Store'].map(store_pyear)
test['Store_PromoInterval'] = test['Store'].map(store_pi)


train['Store_Assortment'].unique()
train['Store_Assortment'].replace({'a':1, 'b':2, 'c':3}, inplace = True)
test['Store_Assortment'].replace({'a':1, 'b':2, 'c':3}, inplace = True)


a = [0, 'Jan,Apr,Jul,Oct', 'Feb,May,Aug,Nov', 'Mar,Jun,Sept,Dec']
b = [0, 1, 2, 3]

train['Store_PromoInterval'].replace(dict(zip(a, b)), inplace = True)
test['Store_PromoInterval'].replace(dict(zip(a, b)), inplace = True)

train['Store_Type'].replace({'a':1, 'b':2, 'c':3, 'd':4}, inplace = True)
test['Store_Type'].replace({'a':1, 'b':2, 'c':3, 'd':4}, inplace = True)

train.fillna(0, inplace = True)
test.fillna(0, inplace = True)

In [616]:
# Changing Datatype

int8_cols = ['DayOfWeek', 'Open', 'Promo','StateHoliday', 'SchoolHoliday', 
'month', 'day', 'dayofweek','week', 'Store_Type', 'Store_Assortment',
'Store_CompetitionOpenSinceMonth','Store_Promo2', 
'Store_Promo2SinceWeek','Store_PromoInterval']

int16_cols = ['Customers', 'year', 'Store', 'Store_CompetitionOpenSinceYear', 'Store_Promo2SinceYear']

int32_cols = ['Sales', 'Store_distance']

def int8(x):
    train[x] = train[x].astype('int8')
    
    test[x] = test[x].astype('int8')

def int16(x):
    train[x] = train[x].astype('int16')
    if x != 'Customers':
        test[x] = test[x].astype('int16')

def int32(x):
    train[x] = train[x].astype('int32')
    if x != 'Sales':
        test[x] = test[x].astype('int32')

for col in int8_cols:
    int8(col)

for col in int16_cols:
    int16(col)

for col in int32_cols:
    int32(col)

In [617]:
# Dropping Samples with Open=0

index = train[ train['Open'] == 0].index
train.drop(index, inplace = True, axis = 0)

In [506]:
#train['Sales'] = np.sqrt(train['Sales'])

In [618]:
target = 'Sales'
date = 'Date'
customer = 'Customers'

cat_cols = ['Store', 'DayOfWeek', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'year', 'month', 'day', 'dayofweek',
       'week', 'Store_Type', 'Store_Assortment',
       'Store_CompetitionOpenSinceMonth', 'Store_CompetitionOpenSinceYear',
       'Store_Promo2', 'Store_Promo2SinceWeek', 'Store_Promo2SinceYear',
       'Store_PromoInterval']

features = [col for col in train.columns if col not in [target, date, customer]]

In [619]:
trn, val = train_test_split(train, test_size = 0.2, random_state = 1999)

##### Input for model
X_trn, X_val = trn[features], val[features]

##### Target column
y_trn, y_val = trn[target], val[target]

##### Features for test data that we will be predicting
X_test = test[features]

%%time

lr = XGBRegressor(random_state=1)

lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error((y_val), (preds)))

print(f'mean_squared_log_error is : {error}')
#1125

---
# Feature Engineering

In [620]:
def join(train_, test_):
    df = pd.concat([train_, test_], axis = 0).reset_index(drop=True)
    return df

def split(df):
    train_, test_ = df[:train.shape[0]], df[train.shape[0]:].reset_index(drop=True)
    return train_, test_

## 1. Count Features

In [510]:
# Store Count
store_count = train.groupby('Store').count()['Date'].to_dict()
store_count = train.groupby('Store').count()['Date'].to_dict()

train['Store_Count'] = train['Store'].map(store_count)
test['Store_Count'] = test['Store'].map(store_count)

In [511]:
## 2. Total Customers Visted Store

cutomers_per_store = train.groupby('Store')['Customers'].sum().to_dict()

train['Cus_per_Store'] = train['Store'].map(cutomers_per_store)
test['Cus_per_Store'] = test['Store'].map(cutomers_per_store)

## 3. Low Popularity Stores w.r.t Customers < 410

store_cus = train.groupby('Store')['Customers'].mean().to_dict()

train['is_low_popular'] = train['Store'].apply(lambda x: 1 if store_cus[x] < 410 else 0)
test['is_low_popular'] = test['Store'].apply(lambda x: 1 if store_cus[x] < 410 else 0)

## 3. Low Popularity Stores w.r.t Customers < 410

store_sales = train.groupby('Store')['Sales'].mean().to_dict()

train['is_low_sales'] = train['Store'].apply(lambda x: 1 if store_sales[x] < 3800 else 0)
test['is_low_sales'] = test['Store'].apply(lambda x: 1 if store_sales[x] < 3800 else 0)

In [512]:
## 3.  Store's Mean Sales

store_sales = train.groupby('Store')['Sales'].mean().to_dict()

train['Store_Median_Sales'] = train['Store'].apply(lambda x: store_sales[x])
test['Store_Median_Sales'] = test['Store'].apply(lambda x: store_sales[x])

#4.  Total Customers Per Week

week_store = train.groupby(['day', 'Store'])['Customers'].sum().to_dict()

def imputer(x):
    try:
        val = week_store[(x['day'], x['Store'])]
        return val
    except:
        return 0
train['Customers_per_week'] = train.apply(lambda x: imputer(x), axis = 1)
test['Customers_per_week'] = test.apply(lambda x: imputer(x), axis = 1)

#cus_per_store = train.groupby('Store')['Customers'].median().to_dict()
#test.loc[ test['Customers_per_week'] == 0, 'Customers_per_week'] = test.loc[ test['Customers_per_week'] == 0]['Store'].apply(lambda x: cus_per_store[x])

# Try later: Impute missing values of 2013
month_sales = train.groupby(['month', 'Store'])['Sales'].mean().to_dict()

#train['Monthly_Sales'] = train.apply(lambda x: month_sales[(x['month'], x['Store'])], axis = 1)
#test['Monthly_Sales'] = test.apply(lambda x: month_sales[(x['month'], x['Store'])], axis = 1)


def imputer(x):
    try:
        val = month_sales[(x['month'], x['Store'])]
        return val
    except:
        return 0fm 
train['Monthly_Sales'] = train.apply(lambda x: imputer(x), axis = 1)
test['Monthly_Sales'] = test.apply(lambda x: imputer(x), axis = 1)

#cus_per_store = train.groupby('Store')['Customers'].median().to_dict()
#test.loc[ test['Customers_per_week'] == 0, 'Customers_per_week'] = test.loc[ test['Customers_per_week'] == 0]['Store'].apply(lambda x: cus_per_store[x])

# Customers w.r.t. StoreType

cus_type = train.groupby(['Store_Type', 'StateHoliday'])['Customers'].median()

#train['Cus_per_type'] = train.apply(lambda x: cus_type[(x['Store_Type'], x['StateHoliday'])], axis = 1)
#test['Cus_per_type'] = test.apply(lambda x: cus_type[(x['Store_Type'], x['StateHoliday'])], axis = 1)

def imputer(x):
    try:
        val = cus_type[(x['Store_Type'], x['StateHoliday'])]
        return val
    except:
        return 0 
train['Cus_per_type'] = train.apply(lambda x: imputer(x), axis = 1)
test['Cus_per_type'] = test.apply(lambda x: imputer(x), axis = 1)

#cus_per_store = train.groupby('Store')['Customers'].median().to_dict()
#test.loc[ test['Customers_per_week'] == 0, 'Customers_per_week'] = test.loc[ test['Customers_per_week'] == 0]['Store'].apply(lambda x: cus_per_store[x])

In [513]:
# store's mean, max, min sale

mean = train.groupby(['Store'])['Sales'].mean()
median = train.groupby(['Store'])['Sales'].median()
max_ = train.groupby(['Store'])['Sales'].max()
min_ = train.groupby(['Store'])['Sales'].min()

train['Mean_sales'] = train['Store'].apply(lambda x: mean[x])
train['Median_sales'] = train['Store'].apply(lambda x: median[x])
train['Max_sales'] = train['Store'].apply(lambda x: max_[x])
train['Min_sales'] = train['Store'].apply(lambda x: min_[x])

test['Mean_sales'] = test['Store'].apply(lambda x: mean[x])
test['Median_sales'] = test['Store'].apply(lambda x: median[x])
test['Max_sales'] = test['Store'].apply(lambda x: max_[x])
test['Min_sales'] = test['Store'].apply(lambda x: min_[x])

mean = train.groupby(['Store', 'year'])['Sales'].mean()
median = train.groupby(['Store', 'year'])['Sales'].median()
max_ = train.groupby(['Store', 'year'])['Sales'].max()
min_ = train.groupby(['Store', 'year'])['Sales'].min()

def imputer(d, x):
    try:
        val = d[(x['Store'], x['year'])]
        return val
    except:
        return None


train['Mean_sales_month'] = train.apply(lambda x: imputer(mean, x), axis = 1)
train['Median_sales_month'] = train.apply(lambda x: imputer(median, x), axis = 1)
train['Max_sales_month'] = train.apply(lambda x: imputer(max_, x), axis = 1)
train['Min_sales_month'] = train.apply(lambda x: imputer(min_, x), axis = 1)

test['Mean_sales_month'] = test.apply(lambda x: imputer(mean, x), axis = 1)
test['Median_sales_month'] = test.apply(lambda x: imputer(median, x), axis = 1)
test['Max_sales_month'] = test.apply(lambda x: imputer(max_, x), axis = 1)
test['Min_sales_month'] = test.apply(lambda x: imputer(min_, x), axis= 1)

<!-- # store_sales_mean = train.groupby('Store')['Sales'].mean().to_dict()
# store_sales_median = train.groupby('Store')['Sales'].median().to_dict()
# store_sales_min = train.groupby('Store')['Sales'].min().to_dict()
# store_sales_max = train.groupby('Store')['Sales'].max().to_dict()

# test.loc[(test['Mean_sales_month'].isna()), 'Mean_sales_month'] = test[(test['Mean_sales_month'].isna())]['Store'].apply(lambda x:store_sales_mean[x])
# test.loc[(test['Median_sales_month'].isna()), 'Median_sales_month'] = test[(test['Median_sales_month'].isna())]['Store'].apply(lambda x:store_sales_median[x])
# test.loc[(test['Max_sales_month'].isna()), 'Max_sales_month'] = test[(test['Max_sales_month'].isna())]['Store'].apply(lambda x:store_sales_max[x])
# test.loc[(test['Min_sales_month'].isna()), 'Min_sales_month'] = test[(test['Min_sales_month'].isna())]['Store'].apply(lambda x:store_sales_min[x]) -->

In [621]:
features = [col for col in train.columns if col not in [target, date, customer]]

trn, val = train_test_split(train, test_size = 0.2, random_state = 1999)

##### Input for model
X_trn, X_val = trn[features], val[features]

##### Target column
y_trn, y_val = trn[target], val[target]

##### Features for test data that we will be predicting
X_test = test[features]

In [622]:
%%time

lr = XGBRegressor(random_state=1)

lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error((y_val), (preds)))

print(f'mean_squared_log_error is : {error}')
#1074
#937 --> 2nd feat

lr = LGBMRegressor(random_state=1)

lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error((y_val), (preds)))

print(f'mean_squared_log_error is : {error}')
#1074
#937 --> 2nd feat

#834 --> 3rd
#820/959 -->mean, median, min, max sales per store

mean_squared_log_error is : 1125.9399137304747
mean_squared_log_error is : 1701.8725894688628
Wall time: 11 s


In [516]:
%%time
lr = XGBRegressor(random_state=1, n_jobs = -1,)

lr.fit(train[features], train[target])

preds = lr.predict(test[features])
preds = np.abs(preds)
# With eval set and no cat_cols: lb score 1093

Wall time: 20.6 s


In [539]:
%%time

params = {'max_depth': 9, 'n_estimators': 819, 'learning_rate': 0.10187280055433054, 'subsample': 0.5611557685582812, 
          'colsample_bytree': 0.7920162015831868, 'colsample_bylevel': 0.7633175258937133,
          'reg_alpha': 82.97737560347895, 'reg_lambda': 27.377692424253066}

lr = XGBRegressor(random_state=1, n_jobs = -1,**params)
lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error((y_val), (preds)))

print(f'mean_squared_log_error is : {error}')

mean_squared_log_error is : 648.1391637500983
Wall time: 3min 45s


In [540]:
%%time

params = {'max_depth': 9, 'n_estimators': 819, 'learning_rate': 0.10187280055433054, 'subsample': 0.5611557685582812, 
          'colsample_bytree': 0.7920162015831868, 'colsample_bylevel': 0.7633175258937133,
          'reg_alpha': 82.97737560347895, 'reg_lambda': 27.377692424253066}

lr = XGBRegressor(random_state=1, n_jobs = -1,**params)
lr.fit(train[features], train[target])

preds = lr.predict(test[features])
preds = np.abs(preds)

Wall time: 4min 39s


In [546]:
%%time

cat_cols = ['Store', 'Promo']
lr = CatBoostRegressor(random_state=1, cat_features=cat_cols, n_estimators=1000)

lr.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 80)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error((y_val), (preds)))

print(f'mean_squared_log_error is : {error}')

# lr.fit(train[features], train[target], eval_set = [(X_val, y_val)], early_stopping_rounds = 80)

# preds = lr.predict(X_test)
# preds = np.abs(preds)
# # With eval set and no cat_cols: lb score 1093

Learning rate set to 0.147523
0:	learn: 2825.1659356	test: 2808.3296255	best: 2808.3296255 (0)	total: 137ms	remaining: 2m 17s
1:	learn: 2552.9025424	test: 2537.9873793	best: 2537.9873793 (1)	total: 298ms	remaining: 2m 28s
2:	learn: 2326.3198819	test: 2312.5955899	best: 2312.5955899 (2)	total: 437ms	remaining: 2m 25s
3:	learn: 2138.8972406	test: 2126.3766689	best: 2126.3766689 (3)	total: 572ms	remaining: 2m 22s
4:	learn: 1986.6205909	test: 1975.3538168	best: 1975.3538168 (4)	total: 701ms	remaining: 2m 19s
5:	learn: 1860.6329618	test: 1850.1373598	best: 1850.1373598 (5)	total: 848ms	remaining: 2m 20s
6:	learn: 1756.3700127	test: 1746.7193743	best: 1746.7193743 (6)	total: 975ms	remaining: 2m 18s
7:	learn: 1670.9524766	test: 1661.4791906	best: 1661.4791906 (7)	total: 1.14s	remaining: 2m 20s
8:	learn: 1601.5996942	test: 1592.9677663	best: 1592.9677663 (8)	total: 1.3s	remaining: 2m 23s
9:	learn: 1546.1041529	test: 1537.7831056	best: 1537.7831056 (9)	total: 1.43s	remaining: 2m 21s
10:	learn: 

84:	learn: 1026.1915131	test: 1025.8406355	best: 1025.8406355 (84)	total: 12.3s	remaining: 2m 12s
85:	learn: 1024.3021433	test: 1023.9409927	best: 1023.9409927 (85)	total: 12.5s	remaining: 2m 12s
86:	learn: 1022.7968193	test: 1022.5548326	best: 1022.5548326 (86)	total: 12.7s	remaining: 2m 12s
87:	learn: 1021.2063610	test: 1020.8687332	best: 1020.8687332 (87)	total: 12.8s	remaining: 2m 13s
88:	learn: 1019.7455407	test: 1019.5194285	best: 1019.5194285 (88)	total: 13s	remaining: 2m 13s
89:	learn: 1018.7657716	test: 1018.6181782	best: 1018.6181782 (89)	total: 13.2s	remaining: 2m 13s
90:	learn: 1017.6924495	test: 1017.6371829	best: 1017.6371829 (90)	total: 13.3s	remaining: 2m 12s
91:	learn: 1016.0739858	test: 1016.2134086	best: 1016.2134086 (91)	total: 13.4s	remaining: 2m 12s
92:	learn: 1014.5629828	test: 1014.6712693	best: 1014.6712693 (92)	total: 13.6s	remaining: 2m 12s
93:	learn: 1012.4854527	test: 1012.5825219	best: 1012.5825219 (93)	total: 13.7s	remaining: 2m 12s
94:	learn: 1011.071598

171:	learn: 922.3588410	test: 924.5960756	best: 924.5960756 (171)	total: 24.4s	remaining: 1m 57s
172:	learn: 921.7088155	test: 923.9804003	best: 923.9804003 (172)	total: 24.6s	remaining: 1m 57s
173:	learn: 921.1080249	test: 923.2991978	best: 923.2991978 (173)	total: 24.7s	remaining: 1m 57s
174:	learn: 920.4582618	test: 922.6621019	best: 922.6621019 (174)	total: 24.8s	remaining: 1m 57s
175:	learn: 919.6950020	test: 921.9997306	best: 921.9997306 (175)	total: 24.9s	remaining: 1m 56s
176:	learn: 919.2136839	test: 921.5776500	best: 921.5776500 (176)	total: 25.1s	remaining: 1m 56s
177:	learn: 918.7433837	test: 921.1270148	best: 921.1270148 (177)	total: 25.2s	remaining: 1m 56s
178:	learn: 917.8985438	test: 920.2755967	best: 920.2755967 (178)	total: 25.4s	remaining: 1m 56s
179:	learn: 917.1980970	test: 919.7159737	best: 919.7159737 (179)	total: 25.5s	remaining: 1m 56s
180:	learn: 916.7935131	test: 919.3568256	best: 919.3568256 (180)	total: 25.7s	remaining: 1m 56s
181:	learn: 916.0946214	test: 

257:	learn: 874.3311138	test: 878.1574776	best: 878.1574776 (257)	total: 36.3s	remaining: 1m 44s
258:	learn: 873.6632507	test: 877.4841755	best: 877.4841755 (258)	total: 36.4s	remaining: 1m 44s
259:	learn: 873.2599882	test: 877.0360920	best: 877.0360920 (259)	total: 36.5s	remaining: 1m 43s
260:	learn: 872.9608222	test: 876.7659632	best: 876.7659632 (260)	total: 36.7s	remaining: 1m 43s
261:	learn: 872.4795679	test: 876.2472844	best: 876.2472844 (261)	total: 36.8s	remaining: 1m 43s
262:	learn: 872.0426631	test: 875.8059288	best: 875.8059288 (262)	total: 37s	remaining: 1m 43s
263:	learn: 871.5469866	test: 875.3164448	best: 875.3164448 (263)	total: 37.1s	remaining: 1m 43s
264:	learn: 871.1843606	test: 874.9692367	best: 874.9692367 (264)	total: 37.2s	remaining: 1m 43s
265:	learn: 870.7357239	test: 874.4888239	best: 874.4888239 (265)	total: 37.3s	remaining: 1m 43s
266:	learn: 870.1662678	test: 874.0374516	best: 874.0374516 (266)	total: 37.5s	remaining: 1m 43s
267:	learn: 869.6772620	test: 87

342:	learn: 841.8037734	test: 847.4341481	best: 847.4341481 (342)	total: 47.5s	remaining: 1m 30s
343:	learn: 841.3639682	test: 847.0641318	best: 847.0641318 (343)	total: 47.6s	remaining: 1m 30s
344:	learn: 841.0353265	test: 846.7478646	best: 846.7478646 (344)	total: 47.7s	remaining: 1m 30s
345:	learn: 840.7538412	test: 846.6106672	best: 846.6106672 (345)	total: 47.9s	remaining: 1m 30s
346:	learn: 840.4643111	test: 846.3260913	best: 846.3260913 (346)	total: 48s	remaining: 1m 30s
347:	learn: 840.1166967	test: 845.9827943	best: 845.9827943 (347)	total: 48.2s	remaining: 1m 30s
348:	learn: 839.7994322	test: 845.6296695	best: 845.6296695 (348)	total: 48.3s	remaining: 1m 30s
349:	learn: 839.2160093	test: 845.0713892	best: 845.0713892 (349)	total: 48.5s	remaining: 1m 30s
350:	learn: 838.7112064	test: 844.5988086	best: 844.5988086 (350)	total: 48.6s	remaining: 1m 29s
351:	learn: 838.4212541	test: 844.3160165	best: 844.3160165 (351)	total: 48.7s	remaining: 1m 29s
352:	learn: 838.1880185	test: 84

...

599:	learn: 780.7438630	test: 791.3000693	best: 791.3000693 (599)	total: 1m 23s	remaining: 55.3s
600:	learn: 780.5552369	test: 791.1453314	best: 791.1453314 (600)	total: 1m 23s	remaining: 55.2s
601:	learn: 780.3706545	test: 791.0228888	best: 791.0228888 (601)	total: 1m 23s	remaining: 55.1s
602:	learn: 780.1980629	test: 790.8353656	best: 790.8353656 (602)	total: 1m 23s	remaining: 54.9s
603:	learn: 780.0729639	test: 790.8005162	best: 790.8005162 (603)	total: 1m 23s	remaining: 54.8s
604:	learn: 779.9213779	test: 790.6690140	best: 790.6690140 (604)	total: 1m 23s	remaining: 54.6s
605:	learn: 779.7735816	test: 790.5578556	best: 790.5578556 (605)	total: 1m 23s	remaining: 54.5s
606:	learn: 779.6387647	test: 790.4148294	best: 790.4148294 (606)	total: 1m 23s	remaining: 54.3s
607:	learn: 779.4517265	test: 790.2305935	best: 790.2305935 (607)	total: 1m 24s	remaining: 54.2s
608:	learn: 779.2484968	test: 790.0265380	best: 790.0265380 (608)	total: 1m 24s	remaining: 54s
609:	learn: 779.0822546	test: 78

685:	learn: 767.1225904	test: 779.2817773	best: 779.2817773 (685)	total: 1m 34s	remaining: 43.2s
686:	learn: 766.9898082	test: 779.1510420	best: 779.1510420 (686)	total: 1m 34s	remaining: 43.1s
687:	learn: 766.8240212	test: 778.8140807	best: 778.8140807 (687)	total: 1m 34s	remaining: 43s
688:	learn: 766.6664108	test: 778.7010265	best: 778.7010265 (688)	total: 1m 34s	remaining: 42.8s
689:	learn: 766.5519336	test: 778.6145310	best: 778.6145310 (689)	total: 1m 35s	remaining: 42.7s
690:	learn: 766.4135030	test: 778.4634064	best: 778.4634064 (690)	total: 1m 35s	remaining: 42.6s
691:	learn: 766.1858516	test: 778.2099388	best: 778.2099388 (691)	total: 1m 35s	remaining: 42.4s
692:	learn: 765.9733301	test: 778.0210870	best: 778.0210870 (692)	total: 1m 35s	remaining: 42.3s
693:	learn: 765.8724182	test: 777.9193557	best: 777.9193557 (693)	total: 1m 35s	remaining: 42.2s
694:	learn: 765.7586463	test: 777.8133640	best: 777.8133640 (694)	total: 1m 35s	remaining: 42s
695:	learn: 765.6910894	test: 777.

771:	learn: 755.4712269	test: 768.7446560	best: 768.7446560 (771)	total: 1m 46s	remaining: 31.4s
772:	learn: 755.3287040	test: 768.6130475	best: 768.6130475 (772)	total: 1m 46s	remaining: 31.3s
773:	learn: 755.2425374	test: 768.5493001	best: 768.5493001 (773)	total: 1m 46s	remaining: 31.1s
774:	learn: 755.1370158	test: 768.4577845	best: 768.4577845 (774)	total: 1m 46s	remaining: 31s
775:	learn: 755.0259699	test: 768.3847147	best: 768.3847147 (775)	total: 1m 46s	remaining: 30.9s
776:	learn: 754.8705963	test: 768.2215495	best: 768.2215495 (776)	total: 1m 47s	remaining: 30.7s
777:	learn: 754.7847659	test: 768.1577824	best: 768.1577824 (777)	total: 1m 47s	remaining: 30.6s
778:	learn: 754.6953237	test: 768.1889732	best: 768.1577824 (777)	total: 1m 47s	remaining: 30.5s
779:	learn: 754.6122545	test: 768.1212450	best: 768.1212450 (779)	total: 1m 47s	remaining: 30.3s
780:	learn: 754.4257285	test: 767.9191374	best: 767.9191374 (780)	total: 1m 47s	remaining: 30.2s
781:	learn: 754.3488977	test: 76

857:	learn: 744.9668539	test: 759.6229847	best: 759.6229847 (857)	total: 1m 58s	remaining: 19.5s
858:	learn: 744.9055880	test: 759.5598375	best: 759.5598375 (858)	total: 1m 58s	remaining: 19.4s
859:	learn: 744.8210587	test: 759.5243495	best: 759.5243495 (859)	total: 1m 58s	remaining: 19.3s
860:	learn: 744.7026601	test: 759.4500544	best: 759.4500544 (860)	total: 1m 58s	remaining: 19.1s
861:	learn: 744.6099145	test: 759.3473921	best: 759.3473921 (861)	total: 1m 58s	remaining: 19s
862:	learn: 744.4702108	test: 759.2901033	best: 759.2901033 (862)	total: 1m 58s	remaining: 18.8s
863:	learn: 744.3780797	test: 759.1786480	best: 759.1786480 (863)	total: 1m 58s	remaining: 18.7s
864:	learn: 744.2521656	test: 759.0815135	best: 759.0815135 (864)	total: 1m 58s	remaining: 18.6s
865:	learn: 744.1251539	test: 758.9795871	best: 758.9795871 (865)	total: 1m 59s	remaining: 18.4s
866:	learn: 744.0174709	test: 758.9383583	best: 758.9383583 (866)	total: 1m 59s	remaining: 18.3s
867:	learn: 743.9126081	test: 75

943:	learn: 735.9992456	test: 752.0871559	best: 752.0871559 (943)	total: 2m 9s	remaining: 7.68s
944:	learn: 735.8807282	test: 751.9584058	best: 751.9584058 (944)	total: 2m 9s	remaining: 7.54s
945:	learn: 735.7460438	test: 751.8541777	best: 751.8541777 (945)	total: 2m 9s	remaining: 7.4s
946:	learn: 735.5596590	test: 751.6610875	best: 751.6610875 (946)	total: 2m 9s	remaining: 7.27s
947:	learn: 735.4407185	test: 751.5580100	best: 751.5580100 (947)	total: 2m 9s	remaining: 7.13s
948:	learn: 735.3010232	test: 751.5201016	best: 751.5201016 (948)	total: 2m 10s	remaining: 6.99s
949:	learn: 735.2188690	test: 751.4385105	best: 751.4385105 (949)	total: 2m 10s	remaining: 6.85s
950:	learn: 735.0932052	test: 751.3976111	best: 751.3976111 (950)	total: 2m 10s	remaining: 6.71s
951:	learn: 735.0122579	test: 751.2987608	best: 751.2987608 (951)	total: 2m 10s	remaining: 6.58s
952:	learn: 734.9112957	test: 751.1840157	best: 751.1840157 (952)	total: 2m 10s	remaining: 6.44s
953:	learn: 734.7949207	test: 751.08

In [547]:
%%time

cat_cols = ['Store', 'Promo']
lr = CatBoostRegressor(random_state=1, cat_features=cat_cols, n_estimators=1000)


lr.fit(train[features], train[target], eval_set = [(X_val, y_val)], early_stopping_rounds = 80)

preds = lr.predict(X_test)
preds = np.abs(preds)
# # With eval set and no cat_cols: lb score 1093

Learning rate set to 0.153878
0:	learn: 2808.2033007	test: 2794.3393205	best: 2794.3393205 (0)	total: 167ms	remaining: 2m 46s
1:	learn: 2525.2377500	test: 2512.9673308	best: 2512.9673308 (1)	total: 352ms	remaining: 2m 55s
2:	learn: 2292.9988260	test: 2281.8336994	best: 2281.8336994 (2)	total: 519ms	remaining: 2m 52s
3:	learn: 2103.7371976	test: 2093.8097058	best: 2093.8097058 (3)	total: 682ms	remaining: 2m 49s
4:	learn: 1949.0181396	test: 1939.8496939	best: 1939.8496939 (4)	total: 832ms	remaining: 2m 45s
5:	learn: 1825.0615226	test: 1816.5260303	best: 1816.5260303 (5)	total: 980ms	remaining: 2m 42s
6:	learn: 1722.9699957	test: 1715.2292225	best: 1715.2292225 (6)	total: 1.15s	remaining: 2m 43s
7:	learn: 1641.3619587	test: 1633.7976673	best: 1633.7976673 (7)	total: 1.31s	remaining: 2m 42s
8:	learn: 1577.4599851	test: 1570.2135597	best: 1570.2135597 (8)	total: 1.48s	remaining: 2m 43s
9:	learn: 1524.7168874	test: 1517.7547164	best: 1517.7547164 (9)	total: 1.63s	remaining: 2m 41s
10:	learn:

84:	learn: 1018.0108187	test: 1016.0204161	best: 1016.0204161 (84)	total: 13.7s	remaining: 2m 27s
85:	learn: 1016.9242304	test: 1015.0020847	best: 1015.0020847 (85)	total: 13.9s	remaining: 2m 28s
86:	learn: 1015.7224899	test: 1013.7758413	best: 1013.7758413 (86)	total: 14.1s	remaining: 2m 28s
87:	learn: 1014.6578136	test: 1012.6451781	best: 1012.6451781 (87)	total: 14.3s	remaining: 2m 28s
88:	learn: 1013.2483254	test: 1011.1959828	best: 1011.1959828 (88)	total: 14.5s	remaining: 2m 28s
89:	learn: 1011.6495996	test: 1009.6101631	best: 1009.6101631 (89)	total: 14.6s	remaining: 2m 27s
90:	learn: 1008.1545241	test: 1006.0043653	best: 1006.0043653 (90)	total: 14.8s	remaining: 2m 27s
91:	learn: 1006.7856245	test: 1004.6361844	best: 1004.6361844 (91)	total: 14.9s	remaining: 2m 27s
92:	learn: 1005.9993025	test: 1003.8894587	best: 1003.8894587 (92)	total: 15.1s	remaining: 2m 27s
93:	learn: 1003.2046203	test: 1001.0374271	best: 1001.0374271 (93)	total: 15.3s	remaining: 2m 27s
94:	learn: 1001.7375

169:	learn: 918.1909696	test: 916.3903899	best: 916.3903899 (169)	total: 27.3s	remaining: 2m 13s
170:	learn: 917.1812665	test: 915.3855215	best: 915.3855215 (170)	total: 27.5s	remaining: 2m 13s
171:	learn: 916.4546186	test: 914.6955769	best: 914.6955769 (171)	total: 27.7s	remaining: 2m 13s
172:	learn: 915.7226472	test: 913.9473420	best: 913.9473420 (172)	total: 27.9s	remaining: 2m 13s
173:	learn: 915.1353847	test: 913.3361196	best: 913.3361196 (173)	total: 28.1s	remaining: 2m 13s
174:	learn: 914.4622909	test: 912.4549581	best: 912.4549581 (174)	total: 28.3s	remaining: 2m 13s
175:	learn: 914.0228774	test: 911.9596254	best: 911.9596254 (175)	total: 28.4s	remaining: 2m 13s
176:	learn: 913.4086139	test: 911.3318407	best: 911.3318407 (176)	total: 28.6s	remaining: 2m 13s
177:	learn: 912.8256270	test: 910.7675952	best: 910.7675952 (177)	total: 28.8s	remaining: 2m 13s
178:	learn: 912.1891469	test: 910.1142657	best: 910.1142657 (178)	total: 29s	remaining: 2m 13s
179:	learn: 911.4206247	test: 90

256:	learn: 868.0250475	test: 866.0247137	best: 866.0247137 (256)	total: 41.3s	remaining: 1m 59s
257:	learn: 867.3875738	test: 865.6479897	best: 865.6479897 (257)	total: 41.4s	remaining: 1m 59s
258:	learn: 867.1177133	test: 865.4180663	best: 865.4180663 (258)	total: 41.6s	remaining: 1m 58s
259:	learn: 866.8964442	test: 865.1878001	best: 865.1878001 (259)	total: 41.7s	remaining: 1m 58s
260:	learn: 866.5444118	test: 864.8498272	best: 864.8498272 (260)	total: 41.9s	remaining: 1m 58s
261:	learn: 866.1395419	test: 864.4661938	best: 864.4661938 (261)	total: 42s	remaining: 1m 58s
262:	learn: 865.7556611	test: 864.0842382	best: 864.0842382 (262)	total: 42.2s	remaining: 1m 58s
263:	learn: 865.3141208	test: 863.6064815	best: 863.6064815 (263)	total: 42.3s	remaining: 1m 58s
264:	learn: 864.8021269	test: 863.0612327	best: 863.0612327 (264)	total: 42.5s	remaining: 1m 57s
265:	learn: 864.0420835	test: 862.2524686	best: 862.2524686 (265)	total: 42.7s	remaining: 1m 57s
266:	learn: 863.6759627	test: 86

341:	learn: 835.7777873	test: 834.0577292	best: 834.0577292 (341)	total: 54.8s	remaining: 1m 45s
342:	learn: 835.4645175	test: 833.7576596	best: 833.7576596 (342)	total: 55s	remaining: 1m 45s
343:	learn: 835.1093628	test: 833.4050273	best: 833.4050273 (343)	total: 55.1s	remaining: 1m 45s
344:	learn: 834.8989890	test: 833.2264820	best: 833.2264820 (344)	total: 55.3s	remaining: 1m 44s
345:	learn: 834.5323909	test: 832.8075694	best: 832.8075694 (345)	total: 55.4s	remaining: 1m 44s
346:	learn: 834.2241464	test: 832.4993362	best: 832.4993362 (346)	total: 55.6s	remaining: 1m 44s
347:	learn: 833.8915781	test: 832.1434508	best: 832.1434508 (347)	total: 55.7s	remaining: 1m 44s
348:	learn: 833.6206972	test: 831.8505255	best: 831.8505255 (348)	total: 55.9s	remaining: 1m 44s
349:	learn: 833.4605520	test: 831.6964801	best: 831.6964801 (349)	total: 56.1s	remaining: 1m 44s
350:	learn: 833.0358867	test: 831.2323880	best: 831.2323880 (350)	total: 56.2s	remaining: 1m 43s
351:	learn: 832.6992579	test: 83

...

595:	learn: 776.4028177	test: 774.3200744	best: 774.3200744 (595)	total: 1m 35s	remaining: 1m 4s
596:	learn: 776.2753451	test: 774.1813976	best: 774.1813976 (596)	total: 1m 35s	remaining: 1m 4s
597:	learn: 776.0804388	test: 773.9456614	best: 773.9456614 (597)	total: 1m 35s	remaining: 1m 4s
598:	learn: 775.7984394	test: 773.7070589	best: 773.7070589 (598)	total: 1m 35s	remaining: 1m 3s
599:	learn: 775.7080283	test: 773.6168220	best: 773.6168220 (599)	total: 1m 35s	remaining: 1m 3s
600:	learn: 775.5056645	test: 773.4062180	best: 773.4062180 (600)	total: 1m 35s	remaining: 1m 3s
601:	learn: 775.3704248	test: 773.2703033	best: 773.2703033 (601)	total: 1m 35s	remaining: 1m 3s
602:	learn: 775.2029793	test: 773.0972722	best: 773.0972722 (602)	total: 1m 36s	remaining: 1m 3s
603:	learn: 775.0640821	test: 772.9418504	best: 772.9418504 (603)	total: 1m 36s	remaining: 1m 3s
604:	learn: 774.7248142	test: 772.6461211	best: 772.6461211 (604)	total: 1m 36s	remaining: 1m 2s
605:	learn: 774.5430863	test: 

681:	learn: 762.7439930	test: 760.2848150	best: 760.2848150 (681)	total: 1m 48s	remaining: 50.8s
682:	learn: 762.6222159	test: 760.1620862	best: 760.1620862 (682)	total: 1m 49s	remaining: 50.6s
683:	learn: 762.3321932	test: 759.8756967	best: 759.8756967 (683)	total: 1m 49s	remaining: 50.5s
684:	learn: 762.1606864	test: 759.6920961	best: 759.6920961 (684)	total: 1m 49s	remaining: 50.3s
685:	learn: 762.1022057	test: 759.6373890	best: 759.6373890 (685)	total: 1m 49s	remaining: 50.2s
686:	learn: 761.9456508	test: 759.4622911	best: 759.4622911 (686)	total: 1m 49s	remaining: 50s
687:	learn: 761.8135984	test: 759.3304806	best: 759.3304806 (687)	total: 1m 49s	remaining: 49.9s
688:	learn: 761.6179228	test: 759.1410681	best: 759.1410681 (688)	total: 1m 50s	remaining: 49.7s
689:	learn: 761.5172389	test: 759.0240991	best: 759.0240991 (689)	total: 1m 50s	remaining: 49.5s
690:	learn: 761.3885403	test: 758.8547796	best: 758.8547796 (690)	total: 1m 50s	remaining: 49.4s
691:	learn: 761.2244110	test: 75

767:	learn: 751.2247718	test: 748.6094327	best: 748.6094327 (767)	total: 2m 2s	remaining: 37.1s
768:	learn: 751.1356859	test: 748.5444715	best: 748.5444715 (768)	total: 2m 2s	remaining: 36.9s
769:	learn: 750.9439964	test: 748.4798527	best: 748.4798527 (769)	total: 2m 3s	remaining: 36.7s
770:	learn: 750.8434110	test: 748.3472295	best: 748.3472295 (770)	total: 2m 3s	remaining: 36.6s
771:	learn: 750.7442595	test: 748.2267946	best: 748.2267946 (771)	total: 2m 3s	remaining: 36.4s
772:	learn: 750.5958875	test: 748.0646314	best: 748.0646314 (772)	total: 2m 3s	remaining: 36.3s
773:	learn: 750.4510819	test: 747.8975152	best: 747.8975152 (773)	total: 2m 3s	remaining: 36.1s
774:	learn: 750.3602654	test: 747.7971577	best: 747.7971577 (774)	total: 2m 3s	remaining: 36s
775:	learn: 750.2565539	test: 747.7213917	best: 747.7213917 (775)	total: 2m 3s	remaining: 35.8s
776:	learn: 750.1518541	test: 747.6186083	best: 747.6186083 (776)	total: 2m 4s	remaining: 35.6s
777:	learn: 749.9497136	test: 747.4166541	

853:	learn: 741.2471966	test: 738.7525521	best: 738.7525521 (853)	total: 2m 16s	remaining: 23.4s
854:	learn: 741.1498807	test: 738.7400473	best: 738.7400473 (854)	total: 2m 16s	remaining: 23.2s
855:	learn: 741.0529022	test: 738.6229861	best: 738.6229861 (855)	total: 2m 17s	remaining: 23.1s
856:	learn: 740.9858626	test: 738.5375583	best: 738.5375583 (856)	total: 2m 17s	remaining: 22.9s
857:	learn: 740.9296400	test: 738.4496188	best: 738.4496188 (857)	total: 2m 17s	remaining: 22.7s
858:	learn: 740.8074964	test: 738.3049024	best: 738.3049024 (858)	total: 2m 17s	remaining: 22.6s
859:	learn: 740.7030583	test: 738.1892757	best: 738.1892757 (859)	total: 2m 17s	remaining: 22.4s
860:	learn: 740.5762075	test: 738.0743227	best: 738.0743227 (860)	total: 2m 17s	remaining: 22.2s
861:	learn: 740.4079715	test: 737.8832169	best: 737.8832169 (861)	total: 2m 17s	remaining: 22.1s
862:	learn: 740.2698533	test: 737.7392428	best: 737.7392428 (862)	total: 2m 18s	remaining: 21.9s
863:	learn: 740.1435273	test: 

939:	learn: 731.9471164	test: 729.8418453	best: 729.8418453 (939)	total: 2m 30s	remaining: 9.62s
940:	learn: 731.8364336	test: 729.7402726	best: 729.7402726 (940)	total: 2m 30s	remaining: 9.46s
941:	learn: 731.7654341	test: 729.6621427	best: 729.6621427 (941)	total: 2m 31s	remaining: 9.3s
942:	learn: 731.7057303	test: 729.6124831	best: 729.6124831 (942)	total: 2m 31s	remaining: 9.14s
943:	learn: 731.6370599	test: 729.5584489	best: 729.5584489 (943)	total: 2m 31s	remaining: 8.98s
944:	learn: 731.4993892	test: 729.4247056	best: 729.4247056 (944)	total: 2m 31s	remaining: 8.82s
945:	learn: 731.3500915	test: 729.2855556	best: 729.2855556 (945)	total: 2m 31s	remaining: 8.66s
946:	learn: 731.2237491	test: 729.1876377	best: 729.1876377 (946)	total: 2m 31s	remaining: 8.5s
947:	learn: 731.1075012	test: 729.0549308	best: 729.0549308 (947)	total: 2m 32s	remaining: 8.34s
948:	learn: 731.0536861	test: 729.0109583	best: 729.0109583 (948)	total: 2m 32s	remaining: 8.18s
949:	learn: 730.9347214	test: 72

In [548]:

test['Predictions'] = preds
test.loc[ test['Open'] == 0, 'Predictions'] = 0
preds = test['Predictions']

index = [i for i in range(test.shape[0])]

d = list(zip(index, preds))

ss = pd.DataFrame(d, columns = ['index', 'Sales'])

In [549]:
ss.to_csv(path + "\\feat_count_xgb.csv", index = False)

---
# Tuning

## 1. XGB

In [561]:
%%time

import optuna
from optuna.samplers import TPESampler

def create_model(trial):
    max_depth = trial.suggest_int("max_depth", 2, 30)
    n_estimators = trial.suggest_int("n_estimators", 500, 1500)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 1)
    subsample = trial.suggest_uniform('subsample', 0.1, 0.99)
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.1, 0.9)
    colsample_bylevel = trial.suggest_uniform('colsample_bylevel', 0.1, 0.9)
    #num_leaves = trial.suggest_int("num_leaves", 2, 5000)
    #min_child_samples = trial.suggest_int('min_child_samples', 3, 200)
    reg_alpha = trial.suggest_float("reg_alpha", 0.1, 100)
    reg_lambda = trial.suggest_float("reg_lambda", 0.1, 100)
    model = XGBRegressor(
        max_depth = max_depth,
        n_estimators = n_estimators,
        learning_rate=learning_rate, 
        subsample = subsample,
        colsample_bytree = colsample_bytree,
        colsample_bylevel = colsample_bylevel,
        #num_leaves=num_leaves, 
        #min_child_samples=min_child_samples,
        random_state=1,
        n_jobs = -1
    )
    return model

sampler = TPESampler(seed=1)
def objective(trial):
    model = create_model(trial)
    model.fit(X_trn, y_trn, eval_set = [ (X_val, y_val)], early_stopping_rounds = 60, verbose = 100)
    preds = model.predict(X_val)
    score = np.sqrt(mean_squared_error(y_val,preds))
    return score

study = optuna.create_study(direction="minimize", sampler=sampler)
study.optimize(objective, n_trials=40)

xgb_params = study.best_params
xgb_params['random_state'] = 1
xgb = XGBRegressor(**xgb_params)
xgb.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 60, verbose = False)
preds = xgb.predict(X_val)
print('Optimized XGB RMSE', np.sqrt(mean_squared_error(y_val, preds)))

[I 2021-05-05 04:08:17,797] A new study created in memory with name: no-name-7777c3ae-3fa6-4311-b04e-13dffc0a2956


[0]	validation_0-rmse:3223.95996
[100]	validation_0-rmse:1152.48828
[200]	validation_0-rmse:1079.49927
[300]	validation_0-rmse:1063.00903
[342]	validation_0-rmse:1068.02649


[I 2021-05-05 04:08:52,331] Trial 0 finished with value: 1061.1435597746365 and parameters: {'max_depth': 7, 'n_estimators': 735, 'learning_rate': 0.7231212485077365, 'subsample': 0.10010179358743695, 'colsample_bytree': 0.3418660581054718, 'colsample_bylevel': 0.21740471265369044, 'reg_alpha': 9.3246256174029, 'reg_lambda': 18.707395116629325}. Best is trial 0 with value: 1061.1435597746365.


[0]	validation_0-rmse:4855.86523
[100]	validation_0-rmse:731.60254
[118]	validation_0-rmse:733.35455


[I 2021-05-05 04:09:35,958] Trial 1 finished with value: 728.9377180757723 and parameters: {'max_depth': 15, 'n_estimators': 1008, 'learning_rate': 0.40279979948836325, 'subsample': 0.5795468932629877, 'colsample_bytree': 0.4353556115226359, 'colsample_bylevel': 0.6481756003174076, 'reg_alpha': 20.524779748178595, 'reg_lambda': 87.82393189545544}. Best is trial 1 with value: 728.9377180757723.


...

[0]	validation_0-rmse:6267.45410
[100]	validation_0-rmse:1144.26624
[200]	validation_0-rmse:1073.89734
[300]	validation_0-rmse:1032.09070
[400]	validation_0-rmse:1012.50873
[500]	validation_0-rmse:989.54712
[600]	validation_0-rmse:975.10150
[700]	validation_0-rmse:960.63214
[800]	validation_0-rmse:947.39349
[900]	validation_0-rmse:937.29681
[970]	validation_0-rmse:931.23761


[I 2021-05-05 04:19:13,535] Trial 6 finished with value: 931.1748598753545 and parameters: {'max_depth': 4, 'n_estimators': 971, 'learning_rate': 0.22990330206002108, 'subsample': 0.1652941128509136, 'colsample_bytree': 0.47539082115051456, 'colsample_bylevel': 0.17693780835636935, 'reg_alpha': 90.34667963123884, 'reg_lambda': 12.037097751273611}. Best is trial 3 with value: 628.4936662023822.


[0]	validation_0-rmse:7044.24365
[100]	validation_0-rmse:662.60907
[200]	validation_0-rmse:661.61859
[250]	validation_0-rmse:661.64697


[W 2021-05-05 04:29:01,276] Trial 7 failed because of the following error: XGBoostError('bad allocation')
Traceback (most recent call last):
  File "C:\Users\sunil\anaconda3\lib\site-packages\optuna\_optimize.py", line 211, in _run_trial
    value_or_values = func(trial)
  File "<timed exec>", line 32, in objective
  File "C:\Users\sunil\anaconda3\lib\site-packages\xgboost\core.py", line 422, in inner_f
    return f(**kwargs)
  File "C:\Users\sunil\anaconda3\lib\site-packages\xgboost\sklearn.py", line 603, in fit
    callbacks=callbacks)
  File "C:\Users\sunil\anaconda3\lib\site-packages\xgboost\training.py", line 235, in train
    early_stopping_rounds=early_stopping_rounds)
  File "C:\Users\sunil\anaconda3\lib\site-packages\xgboost\training.py", line 149, in _train_internal
    return bst.copy()
  File "C:\Users\sunil\anaconda3\lib\site-packages\xgboost\core.py", line 1169, in copy
    return self.__copy__()
  File "C:\Users\sunil\anaconda3\lib\site-packages\xgboost\core.py", line 11

XGBoostError: bad allocation

In [562]:
%%time

import optuna
from optuna.samplers import TPESampler

def create_model(trial):
    max_depth = trial.suggest_int("max_depth", 2, 30)
    n_estimators = trial.suggest_int("n_estimators", 500, 3000)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 1)
    subsample = trial.suggest_uniform('subsample', 0.1, 0.99)
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.1, 0.9)
    colsample_bylevel = trial.suggest_uniform('colsample_bylevel', 0.1, 0.9)
    #num_leaves = trial.suggest_int("num_leaves", 2, 5000)
    #min_child_samples = trial.suggest_int('min_child_samples', 3, 200)
    reg_alpha = trial.suggest_float("reg_alpha", 0.1, 100)
    reg_lambda = trial.suggest_float("reg_lambda", 0.1, 100)
    model = LGBMRegressor(
        max_depth = max_depth,
        n_estimators = n_estimators,
        learning_rate=learning_rate, 
        subsample = subsample,
        colsample_bytree = colsample_bytree,
        colsample_bylevel = colsample_bylevel,
        #num_leaves=num_leaves, 
        #min_child_samples=min_child_samples,
        random_state=1,
        n_jobs = -1
    )
    return model

sampler = TPESampler(seed=1)
def objective(trial):
    model = create_model(trial)
    model.fit(X_trn, y_trn, eval_set = [ (X_val, y_val)], early_stopping_rounds = 40, verbose = 100, eval_metric = 'rmse')
    preds = model.predict(X_val)
    score = np.sqrt(mean_squared_error(y_val,preds))
    return score

study = optuna.create_study(direction="minimize", sampler=sampler)
study.optimize(objective, n_trials=60)

lgb_params = study.best_params
lgb_params['random_state'] = 1
lgb = LGBMRegressor(**lgb_params)
lgb.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 40, verbose = False)
preds = lgb.predict(X_val)
print('Optimized lgb RMSE', np.sqrt(mean_squared_error(y_val, preds)))

[I 2021-05-05 04:29:01,589] A new study created in memory with name: no-name-7915d17a-27d8-4938-947f-fd86e7b2b14a


Training until validation scores don't improve for 40 rounds
[100]	valid_0's rmse: 870.218	valid_0's l2: 757280
[200]	valid_0's rmse: 802.799	valid_0's l2: 644486
[300]	valid_0's rmse: 761.013	valid_0's l2: 579140
[400]	valid_0's rmse: 736.662	valid_0's l2: 542671
[500]	valid_0's rmse: 719.633	valid_0's l2: 517872
[600]	valid_0's rmse: 708.411	valid_0's l2: 501846
[700]	valid_0's rmse: 698.968	valid_0's l2: 488556
Did not meet early stopping. Best iteration is:
[735]	valid_0's rmse: 696.305	valid_0's l2: 484840


[I 2021-05-05 04:29:09,951] Trial 0 finished with value: 696.3045662897358 and parameters: {'max_depth': 7, 'n_estimators': 735, 'learning_rate': 0.7231212485077365, 'subsample': 0.10010179358743695, 'colsample_bytree': 0.3418660581054718, 'colsample_bylevel': 0.21740471265369044, 'reg_alpha': 9.3246256174029, 'reg_lambda': 18.707395116629325}. Best is trial 0 with value: 696.3045662897358.


[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Training until validation scores don't improve for 40 rounds
[100]	valid_0's rmse: 827.01	valid_0's l2: 683945
[200]	valid_0's rmse: 767.633	valid_0's l2: 589260
[300]	valid_0's rmse: 738.221	valid_0's l2: 544970
[400]	valid_0's rmse: 720.834	valid_0's l2: 519601
[500]	valid_0's rmse: 710.281	valid_0's l2: 504499
[600]	valid_0's rmse: 702.318	valid_0's l2: 493250
[700]	valid_0's rmse: 695.529	valid_0's l2: 483761
[800]	valid_0's rmse: 689.853	valid_0's l2: 475897
[900]	valid_0's rmse: 685.824	valid_0's l2: 470354
[1000]	valid_0's rmse: 684.005	valid_0's l2: 467862
[1100]	valid_0's rmse: 682.496	valid_0's l2: 465801
[1200]	valid_0's rmse: 680.874	valid_0's l2: 463589
[1300]	valid_0's rmse: 679.796	valid_0's l2: 462122
[1400]	valid_0's rmse: 678.992	valid_0's l2: 461030
Early stopping, best iteration is

[I 2021-05-05 04:29:26,333] Trial 1 finished with value: 678.8948464780428 and parameters: {'max_depth': 15, 'n_estimators': 1702, 'learning_rate': 0.8478478075191569, 'subsample': 0.3788134300697249, 'colsample_bytree': 0.5196385276582971, 'colsample_bylevel': 0.4547623150236454, 'reg_alpha': 23.034763651609573, 'reg_lambda': 53.487949503807}. Best is trial 1 with value: 678.8948464780428.


...

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Training until validation scores don't improve for 40 rounds
[100]	valid_0's rmse: 828.631	valid_0's l2: 686629
[200]	valid_0's rmse: 759.049	valid_0's l2: 576155
[300]	valid_0's rmse: 728.978	valid_0's l2: 531409
[400]	valid_0's rmse: 705.958	valid_0's l2: 498377
[500]	valid_0's rmse: 690.101	valid_0's l2: 476240
[600]	valid_0's rmse: 677.8	valid_0's l2: 459413
[700]	valid_0's rmse: 668.839	valid_0's l2: 447346
[800]	valid_0's rmse: 662.076	valid_0's l2: 438345
[900]	valid_0's rmse: 654.681	valid_0's l2: 428607
[1000]	valid_0's rmse: 649.73	valid_0's l2: 422149
[1100]	valid_0's rmse: 644.813	valid_0's l2: 415784
[1200]	valid_0's rmse: 641.696	valid_0's l2: 411774
[1300]	valid_0's rmse: 639.026	valid_0's l2: 408354
[1400]	valid_0's rmse: 636.465	valid_0's l2: 405088
[1500]	valid_0's rmse: 634.213	vali

[I 2021-05-05 04:59:28,832] Trial 58 finished with value: 625.9788587914676 and parameters: {'max_depth': 22, 'n_estimators': 2045, 'learning_rate': 0.3721939144277728, 'subsample': 0.7751685559934505, 'colsample_bytree': 0.8160639846175454, 'colsample_bylevel': 0.514380913156005, 'reg_alpha': 34.171000456871916, 'reg_lambda': 13.15556691904304}. Best is trial 18 with value: 612.475207007456.


[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Training until validation scores don't improve for 40 rounds
[100]	valid_0's rmse: 966.297	valid_0's l2: 933729
[200]	valid_0's rmse: 880.151	valid_0's l2: 774667
[300]	valid_0's rmse: 834.395	valid_0's l2: 696215
[400]	valid_0's rmse: 801.455	valid_0's l2: 642330
[500]	valid_0's rmse: 777.909	valid_0's l2: 605142
[600]	valid_0's rmse: 760.168	valid_0's l2: 577856
[700]	valid_0's rmse: 746.052	valid_0's l2: 556593
[800]	valid_0's rmse: 733.583	valid_0's l2: 538144
[900]	valid_0's rmse: 723.855	valid_0's l2: 523967
[1000]	valid_0's rmse: 714.648	valid_0's l2: 510721
[1100]	valid_0's rmse: 706.24	valid_0's l2: 498776
[1200]	valid_0's rmse: 699.584	valid_0's l2: 489417
[1300]	valid_0's rmse: 693.341	valid_0's l2: 480721
[1400]	valid_0's rmse: 688.446	valid_0's l2: 473958
[1500]	valid_0's rmse: 683.829	va

[I 2021-05-05 04:59:58,589] Trial 59 finished with value: 668.5565722063225 and parameters: {'max_depth': 18, 'n_estimators': 1848, 'learning_rate': 0.10012078335189462, 'subsample': 0.8182619382525824, 'colsample_bytree': 0.7849247745495359, 'colsample_bylevel': 0.41881706834260923, 'reg_alpha': 17.049106971452602, 'reg_lambda': 37.44318741874072}. Best is trial 18 with value: 612.475207007456.


[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Optimized lgb RMSE 612.9962562452332
Wall time: 31min 42s


In [563]:
%%time

import optuna
from optuna.samplers import TPESampler

def create_model(trial):
    depth = trial.suggest_int("depth", 1, 14)
    n_estimators = trial.suggest_int("n_estimators", 500, 3000)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 1)
    rsm = trial.suggest_uniform('rsm', 0.1, 0.99)
    #colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.1, 0.9)
    #colsample_bylevel = trial.suggest_uniform('colsample_bylevel', 0.1, 0.9)
    #num_leaves = trial.suggest_int("num_leaves", 2, 5000)
    #min_child_samples = trial.suggest_int('min_child_samples', 3, 200)
    reg_alpha = trial.suggest_float("reg_alpha", 0.1, 100)
    reg_lambda = trial.suggest_float("reg_lambda", 0.1, 100)
    model = CatBoostRegressor(
        depth = depth,
        n_estimators = n_estimators,
        learning_rate=learning_rate, 
        rsm = rsm,
        #colsample_bytree = colsample_bytree,
        #colsample_bylevel = colsample_bylevel,
        #num_leaves=num_leaves, 
        #min_child_samples=min_child_samples,
        random_state=1,
        verbose=False
    )
    return model

sampler = TPESampler(seed=1)
def objective(trial):
    model = create_model(trial)
    model.fit(X_trn, y_trn, eval_set = [ (X_val, y_val)], early_stopping_rounds = 40, verbose = 100)
    preds = model.predict(X_val)
    score = np.sqrt(mean_squared_error(y_val,preds))
    return score

study = optuna.create_study(direction="minimize", sampler=sampler)
study.optimize(objective, n_trials=25)

cat_params = study.best_params
cat_params['random_state'] = 1
cat = CatBoostRegressor(**cat_params)
cat.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 40, verbose = False)
preds = cat.predict(X_val)
print('Optimized cat RMSE', np.sqrt(mean_squared_error(y_val, preds)))

[I 2021-05-05 05:00:43,730] A new study created in memory with name: no-name-f8cee042-e9ac-4d4b-8895-f02ba69b9f6b


0:	learn: 2196.4957100	test: 2190.4424469	best: 2190.4424469 (0)	total: 56.2ms	remaining: 41.3s
100:	learn: 1000.8142757	test: 1009.7461656	best: 1009.7461656 (100)	total: 4.08s	remaining: 25.6s
200:	learn: 900.6567196	test: 913.4163850	best: 913.4163850 (200)	total: 8.02s	remaining: 21.3s
300:	learn: 854.8367317	test: 869.8618055	best: 869.8618055 (300)	total: 12s	remaining: 17.2s
400:	learn: 819.2115329	test: 837.4267090	best: 837.4267090 (400)	total: 15.9s	remaining: 13.2s
500:	learn: 791.8359647	test: 814.4885992	best: 814.4885992 (500)	total: 19.9s	remaining: 9.3s
600:	learn: 769.0390245	test: 794.4355990	best: 794.4355990 (600)	total: 24s	remaining: 5.35s
700:	learn: 754.2459294	test: 782.5559941	best: 782.5559941 (700)	total: 27.9s	remaining: 1.35s


[I 2021-05-05 05:01:13,595] Trial 0 finished with value: 778.5885577771119 and parameters: {'depth': 6, 'n_estimators': 735, 'learning_rate': 0.7231212485077365, 'rsm': 0.10010179358743695, 'reg_alpha': 30.303024005920797, 'reg_lambda': 14.760913492629594}. Best is trial 0 with value: 778.5885577771119.


734:	learn: 749.6648030	test: 778.5885578	best: 778.5885578 (734)	total: 29.3s	remaining: 0us

bestTest = 778.5885578
bestIteration = 734

0:	learn: 3077.7481435	test: 3059.2464209	best: 3059.2464209 (0)	total: 33.8ms	remaining: 21.2s
100:	learn: 1427.7181782	test: 1421.6306426	best: 1421.6306426 (100)	total: 4.07s	remaining: 21.3s
200:	learn: 1391.5014511	test: 1385.8197252	best: 1385.8197252 (200)	total: 7.74s	remaining: 16.5s
300:	learn: 1379.7409925	test: 1375.0239554	best: 1375.0239554 (300)	total: 11.4s	remaining: 12.5s
400:	learn: 1373.3238964	test: 1369.1444235	best: 1369.1444235 (400)	total: 15s	remaining: 8.52s
500:	learn: 1369.3119673	test: 1365.4930183	best: 1365.4930183 (500)	total: 18.8s	remaining: 4.81s
600:	learn: 1366.5363733	test: 1362.9153447	best: 1362.9153447 (600)	total: 22.5s	remaining: 1.05s


[I 2021-05-05 05:01:37,581] Trial 1 finished with value: 1362.3162444792135 and parameters: {'depth': 1, 'n_estimators': 629, 'learning_rate': 0.1943976092638942, 'rsm': 0.4075490470683125, 'reg_alpha': 39.73707067564393, 'reg_lambda': 53.92779172693536}. Best is trial 0 with value: 778.5885577771119.


...

0:	learn: 2565.0752074	test: 2550.3558898	best: 2550.3558898 (0)	total: 65ms	remaining: 2m 56s
100:	learn: 821.9961661	test: 838.4162040	best: 838.4162040 (100)	total: 8.61s	remaining: 3m 43s
200:	learn: 727.9690013	test: 756.5053339	best: 756.5053339 (200)	total: 16.9s	remaining: 3m 31s
300:	learn: 677.2373904	test: 715.9323876	best: 715.9323876 (300)	total: 25.4s	remaining: 3m 24s
400:	learn: 643.4931845	test: 691.0978974	best: 691.0978974 (400)	total: 33.9s	remaining: 3m 16s
500:	learn: 618.6694677	test: 675.2694797	best: 675.2694797 (500)	total: 42.3s	remaining: 3m 7s
600:	learn: 597.7176130	test: 662.6447952	best: 662.6447952 (600)	total: 51s	remaining: 2m 59s
700:	learn: 581.6799750	test: 653.0527870	best: 653.0527870 (700)	total: 59.5s	remaining: 2m 51s
800:	learn: 567.8302017	test: 645.0992584	best: 645.0992584 (800)	total: 1m 8s	remaining: 2m 42s
900:	learn: 555.4053445	test: 638.8391539	best: 638.8391539 (900)	total: 1m 16s	remaining: 2m 34s
1000:	learn: 544.5673444	test: 633

[I 2021-05-05 05:59:49,458] Trial 23 finished with value: 599.9827959584984 and parameters: {'depth': 9, 'n_estimators': 2720, 'learning_rate': 0.25375471616765366, 'rsm': 0.524065637380884, 'reg_alpha': 87.79801367369025, 'reg_lambda': 70.54718555232739}. Best is trial 23 with value: 599.9827959584984.


0:	learn: 2500.6543247	test: 2485.5928007	best: 2485.5928007 (0)	total: 93.4ms	remaining: 3m 26s
100:	learn: 730.6712475	test: 763.8352315	best: 763.8352315 (100)	total: 10.2s	remaining: 3m 32s
200:	learn: 634.2865347	test: 692.1531689	best: 692.1531689 (200)	total: 20s	remaining: 3m 19s
300:	learn: 579.8604511	test: 659.9747794	best: 659.9747794 (300)	total: 30s	remaining: 3m 10s
400:	learn: 545.0130577	test: 643.0122098	best: 643.0122098 (400)	total: 40s	remaining: 3m
500:	learn: 519.4660875	test: 633.4222934	best: 633.4222934 (500)	total: 49.9s	remaining: 2m 49s
600:	learn: 497.3027756	test: 624.8339241	best: 624.8339241 (600)	total: 1m	remaining: 2m 40s
700:	learn: 479.0991879	test: 618.8014552	best: 618.8014552 (700)	total: 1m 9s	remaining: 2m 30s
800:	learn: 464.2694206	test: 614.7853993	best: 614.7853993 (800)	total: 1m 20s	remaining: 2m 20s
900:	learn: 450.4467068	test: 612.0895730	best: 612.0895730 (900)	total: 1m 30s	remaining: 2m 11s
1000:	learn: 437.8847943	test: 609.570883

[I 2021-05-05 06:02:45,373] Trial 24 finished with value: 602.1373294183957 and parameters: {'depth': 11, 'n_estimators': 2207, 'learning_rate': 0.27695977575054354, 'rsm': 0.5041962651404619, 'reg_alpha': 87.63954656070024, 'reg_lambda': 67.47697370044297}. Best is trial 23 with value: 599.9827959584984.


TypeError: __init__() got an unexpected keyword argument 'reg_alpha'

# Cross Validation

In [565]:
cat_params = {'depth': 9, 'n_estimators': 2720, 'learning_rate': 0.25375471616765366,
              'rsm': 0.524065637380884,'reg_lambda': 70.54718555232739}
cat = CatBoostRegressor(random_state = 1, **cat_params)

In [583]:
cat_params = {'depth': 9, 'n_estimators': 2720, 'learning_rate': 0.25375471616765366,
              'rsm': 0.524065637380884,'reg_lambda': 70.54718555232739}
cat_2 = CatBoostRegressor(random_state = 1,cat_features = ['Store', 'Promo'], **cat_params)

In [571]:
lgb_params = {'max_depth': 19, 'n_estimators': 2975, 'learning_rate': 0.23479563219782487, 
              'subsample': 0.6669019444682756, 'colsample_bytree': 0.8910004819287553,'num_leaves' : 200,
              'reg_alpha': 25.932205148585854, 'reg_lambda': 0.24707010414012526}
lgb = LGBMRegressor(random_state=1, **lgb_params)

In [567]:
xgb_params = {'max_depth': 9, 'n_estimators': 819, 'learning_rate': 0.10187280055433054, 
              'subsample': 0.5611557685582812, 'colsample_bytree': 0.7920162015831868, 
              'colsample_bylevel': 0.7633175258937133, 'reg_alpha': 82.97737560347895, 'reg_lambda': 27.377692424253066}
xgb = XGBRegressor(random_state = 1, **xgb_params)

In [579]:
# Cross Validation for Boosting
def cross_val(regressor, train, test, features, name):
    N_splits = 5
    
    oofs = np.zeros(len(train))
    preds = np.zeros(len(test))
    
    target_col = train[target]
    
    folds = StratifiedKFold(n_splits = N_splits, shuffle = True,random_state = 1999)
    stratified_target = pd.qcut( train[target], 10, labels=False, duplicates='drop')
    for index, (trn_idx, val_idx) in enumerate(folds.split(train, stratified_target)):
        print(f'\n================================Fold{index + 1}===================================')
        
        #### Train Set
        X_trn, y_trn = train[features].iloc[trn_idx], train[target].iloc[trn_idx]
        
        #### Validation Set
        X_val, y_val = train[features].iloc[val_idx], train[target].iloc[val_idx]
        
        #### Test Set
        X_test = test[features]
        
        ############ Fitting #############
        _ = regressor.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 60, verbose = 100)
        
        ############ Predicting #############
        val_preds = np.abs(regressor.predict(X_val))
        test_preds = np.abs(regressor.predict(X_test))
        
        error = np.sqrt(mean_squared_error(y_val, val_preds))
        print(f'\n Root Mean Squared Error for Validation set is : {error}')
        
        oofs[val_idx] = val_preds
        preds += test_preds / N_splits
        
    total_error = np.sqrt(mean_squared_error(target_col, oofs))
    print(f'\n\Root Mean Squared Error for oofs is {total_error}')
    
    return oofs, preds

In [576]:
%%time
lgb_oofs, lgb_preds = cross_val(lgb, train, test, features, 'lgb')


================================Fold1===================================
Training until validation scores don't improve for 60 rounds
[200]	valid_0's rmse: 663.765	valid_0's l2: 440583
[400]	valid_0's rmse: 635.667	valid_0's l2: 404073
[600]	valid_0's rmse: 624.492	valid_0's l2: 389990
[800]	valid_0's rmse: 620.377	valid_0's l2: 384868
[1000]	valid_0's rmse: 617.719	valid_0's l2: 381577
[1200]	valid_0's rmse: 616.582	valid_0's l2: 380173
Early stopping, best iteration is:
[1306]	valid_0's rmse: 616.265	valid_0's l2: 379782

 Root Mean Squared Error for Validation set is : 616.264632821031

================================Fold2===================================
Training until validation scores don't improve for 60 rounds
[200]	valid_0's rmse: 653.292	valid_0's l2: 426790
[400]	valid_0's rmse: 623.398	valid_0's l2: 388625
[600]	valid_0's rmse: 612.652	valid_0's l2: 375342
[800]	valid_0's rmse: 606.992	valid_0's l2: 368439
[1000]	valid_0's rmse: 604.446	valid_0's l2: 365355
[1200]	valid

In [577]:
%%time
xgb_oofs, xgb_preds = cross_val(xgb, train, test, features, 'xgb')


================================Fold1===================================
[0]	validation_0-rmse:7015.91406
[200]	validation_0-rmse:755.67450
[400]	validation_0-rmse:696.70782
[600]	validation_0-rmse:671.10193
[800]	validation_0-rmse:655.69531
[818]	validation_0-rmse:654.54309

 Root Mean Squared Error for Validation set is : 654.5432680230269

================================Fold2===================================
[0]	validation_0-rmse:7016.25049
[200]	validation_0-rmse:741.09778
[400]	validation_0-rmse:682.91968
[600]	validation_0-rmse:656.85876
[800]	validation_0-rmse:641.06433
[818]	validation_0-rmse:639.98114

 Root Mean Squared Error for Validation set is : 639.9813898617849

================================Fold3===================================
[0]	validation_0-rmse:7007.70215
[200]	validation_0-rmse:740.74060
[400]	validation_0-rmse:676.87231
[600]	validation_0-rmse:650.95544
[800]	validation_0-rmse:636.32263
[818]	validation_0-rmse:635.27905

 Root Mean Squared Error for Val

In [580]:
%%time
cat_oofs, cat_preds = cross_val(cat, train, test, features, 'cat')


================================Fold1===================================
0:	learn: 2588.5777942	test: 2598.3732218	best: 2598.3732218 (0)	total: 69ms	remaining: 3m 7s
100:	learn: 872.8997244	test: 891.1360953	best: 891.1360953 (100)	total: 8.13s	remaining: 3m 30s
200:	learn: 778.8776638	test: 804.6466032	best: 804.6466032 (200)	total: 16.3s	remaining: 3m 24s
300:	learn: 732.8852870	test: 763.6729913	best: 763.6729913 (300)	total: 24.9s	remaining: 3m 19s
400:	learn: 701.6018228	test: 737.5411413	best: 737.5411413 (400)	total: 33.1s	remaining: 3m 11s
500:	learn: 675.6997139	test: 716.8934143	best: 716.8934143 (500)	total: 41s	remaining: 3m 1s
600:	learn: 661.2622755	test: 705.8399511	best: 705.8399511 (600)	total: 48.2s	remaining: 2m 49s
700:	learn: 647.7475151	test: 695.8048122	best: 695.8048122 (700)	total: 55.8s	remaining: 2m 40s
800:	learn: 633.4595554	test: 686.1466600	best: 686.1466600 (800)	total: 1m 3s	remaining: 2m 32s
900:	learn: 620.2811338	test: 676.9400581	best: 676.9400581

2100:	learn: 544.9433535	test: 612.2133251	best: 612.2133251 (2100)	total: 2m 45s	remaining: 48.8s
2200:	learn: 540.6919877	test: 610.3175419	best: 610.3175419 (2200)	total: 2m 53s	remaining: 40.9s
2300:	learn: 536.7970191	test: 608.7312422	best: 608.7312422 (2300)	total: 3m 1s	remaining: 33s
2400:	learn: 533.0700602	test: 607.3064320	best: 607.3064320 (2400)	total: 3m 8s	remaining: 25.1s
2500:	learn: 529.0136333	test: 605.5902178	best: 605.5902178 (2500)	total: 3m 16s	remaining: 17.2s
2600:	learn: 525.2496892	test: 603.9797669	best: 603.9797669 (2600)	total: 3m 24s	remaining: 9.37s
2700:	learn: 521.7306703	test: 602.6676670	best: 602.6676670 (2700)	total: 3m 32s	remaining: 1.5s
2719:	learn: 521.1095064	test: 602.4168609	best: 602.4168609 (2719)	total: 3m 34s	remaining: 0us

bestTest = 602.4168609
bestIteration = 2719


 Root Mean Squared Error for Validation set is : 602.4168609121801

================================Fold4===================================
0:	learn: 2587.1657950	test

In [584]:
%%time
cat_2_oofs, cat_2_preds = cross_val(cat_2, train, test, features, 'cat_2')


================================Fold1===================================
0:	learn: 2575.3362059	test: 2584.0898417	best: 2584.0898417 (0)	total: 161ms	remaining: 7m 18s
100:	learn: 864.6955729	test: 879.6069617	best: 879.6069617 (100)	total: 16s	remaining: 6m 54s
200:	learn: 775.6896473	test: 798.4332311	best: 798.4332311 (200)	total: 32.1s	remaining: 6m 42s
300:	learn: 729.7596465	test: 757.9615553	best: 757.9615553 (300)	total: 47s	remaining: 6m 17s
400:	learn: 700.5797951	test: 733.7361743	best: 733.7361743 (400)	total: 1m 2s	remaining: 5m 59s
500:	learn: 678.1173685	test: 716.1042302	best: 716.1042302 (500)	total: 1m 17s	remaining: 5m 43s
600:	learn: 660.3052656	test: 702.6708746	best: 702.6708746 (600)	total: 1m 32s	remaining: 5m 27s
700:	learn: 646.2423619	test: 692.9951450	best: 692.9951450 (700)	total: 1m 47s	remaining: 5m 10s
800:	learn: 633.5708330	test: 683.8148063	best: 683.8148063 (800)	total: 2m 2s	remaining: 4m 54s
900:	learn: 622.5208030	test: 676.2595311	best: 676.259

2100:	learn: 551.6589576	test: 616.5812919	best: 616.5812919 (2100)	total: 5m 38s	remaining: 1m 39s
2200:	learn: 547.5546929	test: 614.6834033	best: 614.6834033 (2200)	total: 5m 53s	remaining: 1m 23s
2300:	learn: 543.5640834	test: 613.0419906	best: 613.0419906 (2300)	total: 6m 8s	remaining: 1m 7s
2400:	learn: 540.0853511	test: 611.7096810	best: 611.7096810 (2400)	total: 6m 24s	remaining: 51.1s
2500:	learn: 536.3708095	test: 610.1244910	best: 610.1244910 (2500)	total: 6m 40s	remaining: 35.1s
2600:	learn: 532.8879893	test: 608.6197296	best: 608.6197296 (2600)	total: 6m 56s	remaining: 19.1s
2700:	learn: 529.5688355	test: 607.3897974	best: 607.3897974 (2700)	total: 7m 12s	remaining: 3.04s
2719:	learn: 529.0027046	test: 607.1826327	best: 607.1807535 (2717)	total: 7m 15s	remaining: 0us

bestTest = 607.1807535
bestIteration = 2717

Shrink model to first 2718 iterations.

 Root Mean Squared Error for Validation set is : 607.180753495472

================================Fold4===================

In [590]:
# Cross Validation for Boosting
def lgb_cross_val(regressor, train, test, features, params):
    N_splits = 5
    
    oofs = np.zeros(len(train))
    preds = np.zeros(len(test))
    
    target_col = train[target]
    
    folds = StratifiedKFold(n_splits = N_splits, shuffle = True,random_state = 1999)
    stratified_target = pd.qcut( train[target], 10, labels=False, duplicates='drop')
    for index, (trn_idx, val_idx) in enumerate(folds.split(train, stratified_target)):
        print(f'\n================================Fold{index + 1}===================================')
        
        #### Train Set
        X_trn, y_trn = train[features].iloc[trn_idx], train[target].iloc[trn_idx]
        
        #### Validation Set
        X_val, y_val = train[features].iloc[val_idx], train[target].iloc[val_idx]
        
        #### Test Set
        X_test = test[features]
        
        
        ## Pre Model
        pre_model = LGBMRegressor(random_state = 1, **params)
        pre_model.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 60, verbose = 100, eval_metric = 'rmse')
        ############ Fitting #############
        params2 = params.copy()
        params2['learning_rate'] = params['learning_rate'] * 0.1
        regressor = LGBMRegressor(random_state = 1, **params2)
        _ = regressor.fit(X_trn, y_trn, eval_set = [(X_val, y_val)],
                          early_stopping_rounds = 60, verbose = 100,
                         init_model = pre_model, eval_metric = 'rmse')
        
        ############ Predicting #############
        val_preds = np.abs(regressor.predict(X_val))
        test_preds = np.abs(regressor.predict(X_test))
        
        error = np.sqrt(mean_squared_error(y_val, val_preds))
        print(f'\n Root Mean Squared Error for Validation set is : {error}')
        
        oofs[val_idx] = val_preds
        preds += test_preds / N_splits
        
    total_error = np.sqrt(mean_squared_error(target_col, oofs))
    print(f'\n\Root Mean Squared Error for oofs is {total_error}')
    
    return oofs, preds

In [591]:
%%time
lgb_k_oofs, lgb_k_preds = lgb_cross_val(lgb, train, test, features, lgb_params)


================================Fold1===================================
Training until validation scores don't improve for 60 rounds
[100]	valid_0's rmse: 707.696	valid_0's l2: 500834
[200]	valid_0's rmse: 663.765	valid_0's l2: 440583
[300]	valid_0's rmse: 645.343	valid_0's l2: 416468
[400]	valid_0's rmse: 635.667	valid_0's l2: 404073
[500]	valid_0's rmse: 629.509	valid_0's l2: 396282
[600]	valid_0's rmse: 624.492	valid_0's l2: 389990
[700]	valid_0's rmse: 622.074	valid_0's l2: 386977
[800]	valid_0's rmse: 620.377	valid_0's l2: 384868
[900]	valid_0's rmse: 618.899	valid_0's l2: 383035
[1000]	valid_0's rmse: 617.719	valid_0's l2: 381577
[1100]	valid_0's rmse: 617.117	valid_0's l2: 380833
[1200]	valid_0's rmse: 616.582	valid_0's l2: 380173
[1300]	valid_0's rmse: 616.368	valid_0's l2: 379910
Early stopping, best iteration is:
[1306]	valid_0's rmse: 616.265	valid_0's l2: 379782
Training until validation scores don't improve for 60 rounds
Early stopping, best iteration is:
[1332]	valid_0'

In [602]:
# Cross Validation for Boosting
def normal_cross_val(regressor, train, test, features, name):
    N_splits = 5
    
    oofs = np.zeros(len(train))
    preds = np.zeros(len(test))
    
    target_col = train[target]
    
    folds = StratifiedKFold(n_splits = N_splits, shuffle = True,random_state = 1999)
    stratified_target = pd.qcut( train[target], 10, labels=False, duplicates='drop')
    for index, (trn_idx, val_idx) in enumerate(folds.split(train, stratified_target)):
        print(f'\n================================Fold{index + 1}===================================')
        
        #### Train Set
        X_trn, y_trn = train[features].iloc[trn_idx], train[target].iloc[trn_idx]
        
        #### Validation Set
        X_val, y_val = train[features].iloc[val_idx], train[target].iloc[val_idx]
        
        #### Test Set
        X_test = test[features]
        
        ############ Fitting #############
        _ = regressor.fit(X_trn, y_trn)
        
        ############ Predicting #############
        val_preds = np.abs(regressor.predict(X_val))
        test_preds = np.abs(regressor.predict(X_test))
        
        error = np.sqrt(mean_squared_error(y_val, val_preds))
        print(f'\n Root Mean Squared Error for Validation set is : {error}')
        
        oofs[val_idx] = val_preds
        preds += test_preds / N_splits
        
    total_error = np.sqrt(mean_squared_error(target_col, oofs))
    print(f'\n\Root Mean Squared Error for oofs is {total_error}')
    
    return oofs, preds

In [599]:
test[target] = None

train_new = train[[target, 'Store']].copy()
test_new = test[[target, 'Store']].copy()

train_new['lgb_1'] = lgb_oofs
test_new['lgb_1'] = lgb_preds

train_new['lgb_2'] = lgb_k_oofs
test_new['lgb_2'] = lgb_k_preds

train_new['xgb'] = xgb_oofs
test_new['xgb'] = xgb_preds

train_new['cat'] = cat_oofs
test_new['cat'] = cat_preds

train_new['cat_2'] = cat_2_oofs
test_new['cat_2'] = cat_2_preds

In [600]:
ens_features = [c for c in train_new.columns if c not in [target, 'Store']]

In [607]:
%%time
level_1_lgb_oofs, level_1_lgb_preds = normal_cross_val(LGBMRegressor(), train_new, test_new, ens_features, 'lr')


================================Fold1===================================

 Root Mean Squared Error for Validation set is : 627.114424920179

================================Fold2===================================

 Root Mean Squared Error for Validation set is : 614.6042775932685

================================Fold3===================================

 Root Mean Squared Error for Validation set is : 608.7003163692752

================================Fold4===================================

 Root Mean Squared Error for Validation set is : 607.4606578701178

================================Fold5===================================

 Root Mean Squared Error for Validation set is : 615.1850671852678

\Root Mean Squared Error for oofs is 614.6524516846856
Wall time: 7.77 s


In [608]:

test['Predictions'] = level_1_lgb_preds
test.loc[ test['Open'] == 0, 'Predictions'] = 0
preds = test['Predictions']

index = [i for i in range(test.shape[0])]

d = list(zip(index, preds))

ss = pd.DataFrame(d, columns = ['index', 'Sales'])

In [609]:
ss.to_csv(path + "\\ens.csv", index = False)

In [626]:
train[ train['Store'] == 1].describe().T

,count,mean,std,min,25%,50%,75%,max
Store,554.0,1.000000,0.000000,1.0,1.00,1.0,1.0,1.0
DayOfWeek,554.0,3.498195,1.716054,1.0,2.00,3.0,5.0,6.0
Sales,554.0,4716.821300,1047.665103,2362.0,3919.25,4599.5,5284.5,9528.0
Customers,554.0,555.579422,96.259135,298.0,493.00,542.0,599.0,1130.0
Open,554.0,1.000000,0.000000,1.0,1.00,1.0,1.0,1.0
Promo,554.0,0.447653,0.497702,0.0,0.00,0.0,1.0,1.0
StateHoliday,554.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0
SchoolHoliday,554.0,0.193141,0.395119,0.0,0.00,0.0,0.0,1.0
year,554.0,2014.178700,0.649535,2013.0,2014.00,2014.0,2015.0,2015.0
month,554.0,6.341155,3.551360,1.0,3.00,6.0,10.0,12.0


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,dayofweek,week,Store_Type,Store_Assortment,Store_distance,Store_CompetitionOpenSinceMonth,Store_CompetitionOpenSinceYear,Store_Promo2,Store_Promo2SinceWeek,Store_Promo2SinceYear,Store_PromoInterval
619155,1,1,2013-12-23,9528,1130,1,0,0,1,2013,12,23,0,52,3,1,1270,9,2008,0,0,0,0
